In [1]:
import pandas as pd
import numpy as np
import generatedata as gd

In [2]:
param1 = (1.7, -0.5, -6)

In [3]:
# index_col = 0にしないとunnamedがついてくる
df = pd.read_csv("1.7--0.5--6.csv", index_col = 0)

## 教師ありでは均衡のタイプを予測できるか？

結構予測できる

In [4]:
# 一つしか均衡がないかったやつをsingleというダミー変数で表す
df["single"] = df["equitype"].apply(lambda x:1 if x == 0.0 else 0)

In [24]:
df.head(5)

,pop,dist1,dist2,entryprob1,entryprob2,equitype,entry1,entry2,single
0,1.785304,2.490992,0.834502,0.023052,0.922701,0.0,0,1,1
1,2.147697,2.314002,0.919603,0.845843,0.131954,2.0,1,0,0
2,1.941536,2.530796,1.679463,0.032224,0.906250,1.0,0,1,0
3,1.010513,1.382964,1.607856,0.689004,0.038487,2.0,1,1,0
4,1.133806,1.769167,1.178164,0.028299,0.763185,1.0,0,1,0


In [5]:
# type 0かそうでないかの予測
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df.drop(["entryprob1", "entryprob2", "equitype","single"], axis = 1)
y = df.single

clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
# 係数
coeff_df = pd.DataFrame([X.columns, clf.coef_[0]]).T
coeff_df

,0,1
0,pop,-0.132387
1,dist1,1.26925
2,dist2,0.0531488
3,entry1,-0.128074
4,entry2,0.191488


In [7]:
predicted_value = clf.predict(X)

In [8]:
# そこそこ予測できるな？
accuracy_score(y, predicted_value)

0.70480752173141747

testとtrainに分けてみる

In [11]:
# これでもかなり予想できる
from sklearn.model_selection import train_test_split
X = df.drop(["entryprob1", "entryprob2", "equitype","single"], axis = 1)
y = df.single
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf_train = LogisticRegression()
clf_train.fit(X_train, y_train)

pred = clf_train.predict(X_test)
accuracy_score(y_test, pred)

0.71394799054373526

In [12]:
coeff_df = pd.DataFrame([X.columns, clf_train.coef_[0]]).T
coeff_df

,0,1
0,pop,-0.158694
1,dist1,1.2634
2,dist2,-0.017224
3,entry1,-0.0282272
4,entry2,0.239185


係数としては、dist1とentry2がでかい。今のパラメータはstrategic substituteなので、企業1が参入しにくい状態であればsingeになりやすいと予測してることになるね。

## 教師なしの分類タスクで予想できるか？

クラスター数を指定すると、その想定通りに予想できてる。

In [25]:
# k-means （クラスター数3）
# 一気に3タイプの均衡を分類する
from sklearn.cluster import KMeans
X = df.drop(["entryprob1", "entryprob2", "equitype","single"], axis = 1)

k_means = KMeans(n_clusters = 3)
k_means.fit(X)
df["k-means-pred"] = pd.DataFrame(k_means.labels_)
pd.crosstab(df["equitype"], df["k-means-pred"])

k-means-pred,0,1,2
equitype,,,
0.0,1110,783,491
1.0,197,275,1185
2.0,29,1443,124


In [27]:
# k-means （クラスター数2）
# singleかどうかを分類する
k_means2 = KMeans(n_clusters = 2)
k_means2.fit(X)
df["k-means2-pred"] = pd.DataFrame(k_means2.labels_)
pd.crosstab(df["single"], df["k-means2-pred"])

k-means2-pred,0,1
single,,
0,226,3027
1,1110,1274


In [28]:
pd.crosstab(df["k-means-pred"], df["k-means2-pred"])

k-means2-pred,0,1
k-means-pred,,
0,1336,0
1,0,2501
2,0,1800


最後のやつから、このサンプルデータでk-meansによって推定された均衡選択メカニズムは25:18だって言える。（本当は1:1）

また、通常の意味での均衡選択メカニズムを考える前の段階として、単一の均衡しか存在しない市場だっていうのもデータからかなりの精度でわかるということがわかった。